In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import scipy.stats as stats

In [ ]:
#NBA Positions
URL = "https://www.cbssports.com/nba/stats/player/scoring/nba/regular/all-pos/qualifiers/?sortcol=gp&sortdir=descending&page=1"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

name = []
positions = []
positions_binary = []

results = soup.find_all("span", class_ = "CellPlayerName--long")
for line in results:
    player_name = line.find('a')
    player_position = (line.find('span', class_ = "CellPlayerName-position")).text
    player_position = player_position.replace("\n                            ", "")
    player_position = player_position.replace("\n                        ", "")
    name.append(player_name.text)
    positions.append(player_position)

URL = URL.replace("1", "")

for i in range(2,13):
    URL = URL.replace(str(i - 1), "")
    URL = URL + str(i)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    results = soup.find_all("span", class_ = "CellPlayerName--long")
    for line in results:
        player_name = line.find('a')
        player_position = (line.find('span', class_ = "CellPlayerName-position")).text
        player_position = player_position.replace("\n                            ", "")
        player_position = player_position.replace("\n                        ", "")
        name.append(player_name.text)
        positions.append(player_position)

# POSSIBLE POSITIONS: C, PF, SF, PG, SG

#INTERACTED REGRESSION VARIABLE
#Also we should find the total points scored among players or avg ppg and group by position to see the highest scoring positions
for i in positions:
    if (i == "SG" or i == "PG" or i == "SF"):
        positions_binary.append(1)
    else:
        positions_binary.append(0)

In [ ]:
#NBA Performance Stats
URL = "https://www.cbssports.com/nba/stats/player/scoring/nba/regular/all-pos/qualifiers/?sortcol=gp&sortdir=descending&page=1"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

data = []
usable_data = []

starts  = []
games   = []
minutes = []
mpg     = []
points  = []
ppg     = []
fgper   = []

results = soup.find_all("td", class_ = "TableBase-bodyTd")
for number in results:
    data.append(number.text)

URL = URL.replace("1", "")

for i in range(2,13):
    URL = URL.replace(str(i - 1), "")
    URL = URL + str(i)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    results = soup.find_all("td", class_ = "TableBase-bodyTd")
    for number in results:
        data.append(number.text) 

#USABLE DATA
for i in range(len(data)):
    if ((i != 0) and (i%14 != 0)):
        stat = data[i].replace("\n                    ", "")
        stat = stat.replace("                 ", "")
        if ("—" in data[i]):
            stat = 0
        usable_data.append(float(stat))

#START PERCENTAGE
for i in range(int(len(usable_data)/13)):
    gp = usable_data[i*13]
    gs = usable_data[(i*13) + 1]
    prop = gs/gp
    starts.append(prop)

#GAMES PLAYED
for i in range(int(len(usable_data)/13)):
    games.append(int(usable_data[i*13]))

#MINUTES PER GAME
for i in range(int(len(usable_data)/13)):
    mpg.append(float(usable_data[(i*13) + 2]))

#TOTAL MINUTES
for i in range(int(len(usable_data)/13)):
    minutes_pg = usable_data[(i*13) + 2]
    games_played = usable_data[i*13]
    minutes.append(float(minutes_pg*games_played))

#POINTS PER GAME
for i in range(int(len(usable_data)/13)):
    ppg.append(float(usable_data[(i*13) + 3]))

#TOTAL POINTS
for i in range(int(len(usable_data)/13)):
    points_pg = usable_data[(i*13) + 3]
    games_played = usable_data[i*13]
    points.append(int(points_pg*games_played))

#FIELD GOAL %
for i in range(int(len(usable_data)/13)):
    fgper.append(float(usable_data[(i*13) + 6]))

In [ ]:
#NBA Salaries
URL = "http://www.espn.com/nba/salaries"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

results = soup.find_all("div", class_ = "mod-content")
table = soup.find_all("table", class_ = "tablehead")

data_even = soup.find_all("tr", class_ = "evenrow")
data_odd = soup.find_all("tr", class_ = "oddrow")

names_even = []
names_odd  = []

salaries_even = []
salaries_odd  = []

#These will be the final columns to be added to the csv
names = []
salaries = []

for line in data_even:
    player_name = line.find('a')
    names_even.append(player_name.text)
    player_salary = line.find('td', style="text-align:right;")
    player_salary = (str(player_salary.text)).replace("$","")
    player_salary = (player_salary).replace(",","")
    salaries_even.append(player_salary)
for line in data_odd:
    player_name = line.find('a')
    names_odd.append(player_name.text)
    player_salary = line.find('td', style="text-align:right;")
    player_salary = (str(player_salary.text)).replace("$","")
    player_salary = (player_salary).replace(",","")
    salaries_odd.append(player_salary)
for i in range(len(names_even)):
    names.append(names_odd[i])
    names.append(names_even[i])
    salaries.append(salaries_odd[i])
    salaries.append(salaries_even[i])

for i in range(2,16):
    if i != 0:
        URL = "http://www.espn.com/nba/salaries/_"
        URL = URL + "/page/" + str(i)

        page = requests.get(URL)
        soup = BeautifulSoup(page.content, "html.parser")

        data_even = soup.find_all("tr", class_ = "evenrow")
        data_odd = soup.find_all("tr", class_ = "oddrow")

        names_even = []
        names_odd  = []

        salaries_even = []
        salaries_odd  = []

    for line in data_even:
        player_name = line.find('a')
        names_even.append(player_name.text)
        player_salary = line.find('td', style="text-align:right;")
        player_salary = (str(player_salary.text)).replace("$","")
        player_salary = (player_salary).replace(",","")
        salaries_even.append(player_salary)
    for line in data_odd:
        player_name = line.find('a')
        names_odd.append(player_name.text)
        player_salary = line.find('td', style="text-align:right;")
        player_salary = (str(player_salary.text)).replace("$","")
        player_salary = (player_salary).replace(",","")
        salaries_odd.append(player_salary)
    for i in range(len(names_even)):
        names.append(names_odd[i])
        names.append(names_even[i])
        salaries.append(salaries_odd[i])
        salaries.append(salaries_even[i])
df = pd.read_csv('NBA_data_salaries.csv')
df["Names"] = names
df["Salary"] = salaries
df.to_csv('NBA_data_salaries.csv')

In [ ]:
#Testing to see if positions have meaningful effect on salary
positions_test1 = []
positions_test2 = []
for i in range(len(positions)):
    test = []
    test.append(positions[i])
    test.append(ppg[i])
    positions_test1.append(test)

for i in range(len(positions)):
    test = []
    test.append(positions[i])
    test.append(points[i])
    positions_test2.append(test)

SF_val = []
C_val = []
PG_val = []
SG_val = []
PF_val = []

for i in range(len(positions_test1)):
    if positions_test1[i][0] == 'SF':
        SF_val.append(positions_test1[i][1])
    elif positions_test1[i][0] == 'C':
        C_val.append(positions_test1[i][1])
    elif positions_test1[i][0] == 'PG':
        PG_val.append(positions_test1[i][1])
    elif positions_test1[i][0] == 'SG':
        SG_val.append(positions_test1[i][1])
    elif positions_test1[i][0] == 'PF':
        PF_val.append(positions_test1[i][1])

SF_val1 = []
C_val1 = []
PG_val1 = []
SG_val1 = []
PF_val1 = []

for i in range(len(positions_test2)):
    if positions_test1[i][0] == 'SF':
        SF_val1.append(positions_test2[i][1])
    elif positions_test1[i][0] == 'C':
        C_val1.append(positions_test2[i][1])
    elif positions_test1[i][0] == 'PG':
        PG_val1.append(positions_test2[i][1])
    elif positions_test1[i][0] == 'SG':
        SG_val1.append(positions_test2[i][1])
    elif positions_test1[i][0] == 'PF':
        PF_val1.append(positions_test2[i][1])

print("PPG grouped by position:          ",  stats.f_oneway(SF_val, C_val, PG_val, SG_val, PF_val))
print("Total Points grouped by position: ", stats.f_oneway(SF_val1, C_val1, PG_val1, SG_val1, PF_val1))

In [ ]:
salaries_sorted = []

for i in range(len(name)):
    salaries_sorted.append(0)

for i in range(len(name)):
    if name[i] in names:
        salaries_sorted[i] = salaries[names.index(name[i])]

In [ ]:
df = pd.read_csv('NBA_data_perf.csv')
df["Name"] = name
df["Position"] = positions
df["Starts"] = starts
df["Total Games"] = games
df["Total Mins"] = minutes
df["Total Points"] = points
df["MPG"] = mpg
df["PPG"] = ppg
df["FG %"] = fgper
df["Salaries"] = salaries_sorted
df.to_csv('NBA_data_perf.csv')

In [ ]:
missing_names = []
missing_salaries = []

for i in range(len(names)):
    if names[i] not in name:
        missing_names.append(names[i])
        missing_salaries.append(salaries[i])
        
df = pd.read_csv('missing.csv')
df["Name"] = missing_names
df["Salary"] = missing_salaries
df.to_csv('missing.csv')


In [ ]:
df = pd.read_csv('nba-player-stats-final.csv')
sal_names = df['Name']
sal_salaries = df['Salaries']

In [ ]:
URL = "https://www.basketball-reference.com/contracts/players.html"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

names1 = []
names2 = []
missing_names = []
missing_salaries = []
salaries_found = []

for i in range(len(sal_names)):
    if sal_salaries[i] == 0:
        names2.append(sal_names[i])
        
results = soup.find_all(attrs = {"data-stat": "y1"})
for i in results:
    if i.text != "2021-22":
        if "$" in i.text:
            salary = (i.text).replace("$","")
            salary = salary.replace(",", "")
            missing_salaries.append(int(salary))
        else:
            missing_salaries.append(0)

line = soup.find_all("tr")


for i in line:
    name = i.find('a')
    if "href" in str(name):
        if name.text in names2:
            missing_names.append(name.text)

results[1]